In [9]:
import pandas as pd
import fastparquet
import glob
import multiprocessing as mp
from multiprocessing import Pool
import numpy as np
import time
import random

In [10]:
num_partitions=round(0.95*mp.cpu_count())
all_context_files=[]
from_path='/datanaco/dharp/compounds/datasets/googleV3/'
to_path='/data/dharp/compounds/datasets/contexts/'
for filename in glob.glob(from_path+'*parq'):
    all_context_files.append(filename)
    
len(all_context_files)

22

In [24]:
def context_maker(df):
    df[['w1','w2','w3','w4','w5']]=df.lemma_pos.str.split(" ",expand=True)

    context_df=pd.melt(df,id_vars=['count'],value_vars=['w1','w2','w3','w4','w5'])
    
    context_df=context_df.groupby(['value'],observed=True)['count'].sum().to_frame()
    context_df.reset_index(inplace=True)
    context_df[['word','pos']]=context_df['value'].str.rsplit('_',expand=True,n=1)
    
    context_df=context_df.groupby(['word','pos'],observed=True)['count'].sum().to_frame()
    context_df.reset_index(inplace=True)
    context_df=context_df.loc[context_df.pos.isin(['NOUN','ADJ','VERB','ADV'])]
    context_df['context']=context_df['word']+"_"+context_df['pos']
    context_df=context_df.groupby(['context'],observed=True)['count'].sum().to_frame()
    context_df.reset_index(inplace=True)

    return context_df    

In [12]:
def parquet_file_processer(f):
    parquet_context_df_list=[]
    print(f'File num: {f} name: {f}')
    cur_parq=fastparquet.ParquetFile(f)
    print(f'Number of groups: {len(cur_parq.row_groups)}')
    for i,df in enumerate(cur_parq.iter_row_groups()):
        print(f'Group {i+1}')
        cur_time=time.time()
        context_df=context_maker(df)
        print(f'Total time taken for Group {i+1}: {round(time.time()-cur_time)} secs')        
        parquet_context_df_list.append(context_df)
    
    parq_context_df=pd.concat(parquet_context_df_list,ignore_index=True)
    print(parq_context_df.shape[0])

    parq_context_df=parq_context_df.groupby(['context','year'])['count'].sum().to_frame()
    print(parq_context_df.shape[0])
    parq_context_df.reset_index(inplace=True)
    return parq_context_df

In [13]:
f='/datanaco/dharp/compounds/datasets/googleV3/2010.parq'
cur_fname=f.split('.')[0].split('/')[-1]
print(f'Current parquet file: {f}')
cur_parq=fastparquet.ParquetFile(f)

Current parquet file: /datanaco/dharp/compounds/datasets/googleV3/2010.parq


In [25]:
df=parquet_file_processer('/datanaco/dharp/compounds/datasets/googleV3/1800.parq')


File num: /datanaco/dharp/compounds/datasets/googleV3/1800.parq name: /datanaco/dharp/compounds/datasets/googleV3/1800.parq
Number of groups: 5
Group 1


ValueError: Columns must be same length as key

In [30]:

first.lemma_pos.str.count(' ').add(1).value_counts(normalize=True)

5     9.990066e-01
10    2.990289e-04
8     2.632266e-04
9     1.364425e-04
7     1.242339e-04
6     8.305020e-05
11    6.240813e-05
12    2.485884e-05
16    6.023951e-08
22    4.015967e-08
37    4.015967e-08
33    2.007984e-08
Name: lemma_pos, dtype: float64

In [26]:
first.lemma_pos.str.split(" ",expand=True)

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,'_AUX,m_AUX,_SPACE,_AUX,,,,,,be,...,None,None,None,None,None,None,None,None,None,None
1,'_AUX,m_AUX,_SPACE,_AUX,,,,be,,_SPACE,...,None,None,None,None,None,None,None,None,None,None
2,'_AUX,m_AUX,_SPACE,_AUX,,,,can,,_SPACE,...,None,None,None,None,None,None,None,None,None,None
3,'_AUX,m_AUX,_SPACE,_VERB,,,,,,offer,...,None,None,None,None,None,None,None,None,None,None
4,'_AUX,m_AUX,_SPACE,_VERB,,,,have,,_SPACE,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49801200,",_PUNCT",of_ADP,insect_NOUN,and_CCONJ,of_ADP,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
49801201,",_PUNCT",of_ADP,insect_NOUN,and_CCONJ,other_ADJ,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
49801202,",_PUNCT",of_ADP,insect_NOUN,and_CCONJ,their_PRON,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
49801203,",_PUNCT",of_ADP,insect_NOUN,in_ADP,the_PRON,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [14]:
first = next(iter(cur_parq.iter_row_groups()))


In [13]:
def parquet_batch_processer(cur_list):
    parquet_batch_df_list=[]
    
    for cur_file in cur_list:
        parquet_batch_df_list.append(parquet_file_processer(cur_file))
        
    batch_context_df=pd.concat(parquet_batch_df_list,ignore_index=True)
    print(batch_context_df.shape[0])
    
    batch_context_df=batch_context_df.groupby(['context','year'])['count'].sum().to_frame()
    batch_context_df.reset_index(inplace=True)
    print(batch_context_df.shape[0])
    
    return batch_context_df

In [20]:
all_context_files

['/datanaco/dharp/compounds/datasets/googleV3/1860.parq',
 '/datanaco/dharp/compounds/datasets/googleV3/1870.parq',
 '/datanaco/dharp/compounds/datasets/googleV3/1900.parq',
 '/datanaco/dharp/compounds/datasets/googleV3/1850.parq',
 '/datanaco/dharp/compounds/datasets/googleV3/1810.parq',
 '/datanaco/dharp/compounds/datasets/googleV3/1980.parq',
 '/datanaco/dharp/compounds/datasets/googleV3/1830.parq',
 '/datanaco/dharp/compounds/datasets/googleV3/1960.parq',
 '/datanaco/dharp/compounds/datasets/googleV3/1970.parq',
 '/datanaco/dharp/compounds/datasets/googleV3/1990.parq',
 '/datanaco/dharp/compounds/datasets/googleV3/1910.parq',
 '/datanaco/dharp/compounds/datasets/googleV3/1930.parq',
 '/datanaco/dharp/compounds/datasets/googleV3/1880.parq',
 '/datanaco/dharp/compounds/datasets/googleV3/1890.parq',
 '/datanaco/dharp/compounds/datasets/googleV3/1950.parq',
 '/datanaco/dharp/compounds/datasets/googleV3/1840.parq',
 '/datanaco/dharp/compounds/datasets/googleV3/2010.parq',
 '/datanaco/dh

In [11]:
i

0

In [16]:
all_context_df_list=[]
for i,cur_parq_file_list in enumerate(all_context_files_list):
    
    cur_time=time.time()

    print(f'Iteration {i+1}')
    
    pool = Pool(len(cur_parq_file_list))
    print('Started parallelization')
    
    results=pool.map_async(parquet_batch_processer,cur_parq_file_list,chunksize=1)
    pool.close()
    pool.join()
    
    curr_df_list=results.get()
    current_context_df=pd.concat(curr_df_list,ignore_index=True)
    
    print(current_context_df.shape[0])
    
    current_context_df=current_context_df.groupby(['context','year'])['count'].sum().to_frame()
    current_context_df.reset_index(inplace=True)
    print(current_context_df.shape[0])
    
    print(f'Total time taken for Iteration {i+1}: {round(time.time()-cur_time)} secs')        
    all_context_df_list.append(current_context_df)
    
    
final_context_df=pd.concat(all_context_df_list,ignore_index=True)

final_context_df=final_context_df.groupby(['context','year'])['count'].sum().to_frame()
final_context_df.reset_index(inplace=True)
print(final_context_df.shape[0])

#final_context_df.to_pickle(

Iteration 1
File num: / name: /File num: / name: /File num: / name: /File num: / name: /File num: / name: /File num: / name: /File num: / name: /
File num: / name: /File num: / name: /File num: / name: /
File num: / name: /








Started parallelization


Process ForkPoolWorker-32:
Process ForkPoolWorker-25:
Process ForkPoolWorker-33:
Process ForkPoolWorker-27:
Process ForkPoolWorker-29:
Process ForkPoolWorker-24:
Process ForkPoolWorker-30:
Process ForkPoolWorker-23:
Process ForkPoolWorker-31:
Process ForkPoolWorker-28:
Process ForkPoolWorker-26:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):


KeyboardInterrupt: 

Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/data/dharp/packages/miniconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/data/dharp/packages/miniconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/data/dharp/packages/miniconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/data/dharp/packages/miniconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/data/dharp/packages/miniconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/data/dharp/packages/miniconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/data/dharp/packages/miniconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File